# Imports

In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import copy
import matplotlib.pyplot as plt
import math
import os
from tqdm import tqdm

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Changes made in the custom Dataset class for Flowers102 in Pytorch to address the different split that was requested

In [2]:
from pathlib import Path
from typing import Any, Tuple, Callable, Optional

import PIL.Image

from torchvision.datasets.utils import check_integrity, download_and_extract_archive, download_url, verify_str_arg
from torchvision.datasets.vision import VisionDataset

from sklearn.model_selection import train_test_split


class Flowers102(VisionDataset):
    """`Oxford 102 Flower <https://www.robots.ox.ac.uk/~vgg/data/flowers/102/>`_ Dataset.
    Oxford 102 Flower is an image classification dataset consisting of 102 flower categories. The
    flowers were chosen to be flowers commonly occurring in the United Kingdom. Each class consists of
    between 40 and 258 images.

    The images have large scale, pose and light variations. In addition, there are categories that
    have large variations within the category, and several very similar categories.

    Args:
        root (string): Root directory of the dataset.
        split (string, optional): The dataset split, supports ``"train"`` (default), ``"val"``, or ``"test"``.
        transform (callable, optional): A function/transform that takes in an PIL image and returns a
            transformed version. E.g, ``transforms.RandomCrop``.
        target_transform (callable, optional): A function/transform that takes in the target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """

    _download_url_prefix = "https://www.robots.ox.ac.uk/~vgg/data/flowers/102/"
    _file_dict = {  # filename, md5
        "image": ("102flowers.tgz", "52808999861908f626f3c1f4e79d11fa"),
        "label": ("imagelabels.mat", "e0620be6f572b9609742df49c70aed4d"),
        "setid": ("setid.mat", "a5357ecc9cb78c4bef273ce3793fc85c"),
    }
    _splits_map = {"train": "trnid", "val": "valid", "test": "tstid"}

    def __init__(
        self,
        root: str,
        split: str = "train",
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        download: bool = False,
        random_s: int = 123
    ) -> None:
        super().__init__(root, transform=transform, target_transform=target_transform)
        self._split = verify_str_arg(split, "split", ("train", "val", "test"))
        self._base_folder = Path(self.root) / "flowers-102"
        self._images_folder = self._base_folder / "jpg"

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError("Dataset not found or corrupted. You can use download=True to download it")

        from scipy.io import loadmat
        labels = loadmat(self._base_folder / self._file_dict["label"][0], squeeze_me=True)
        image_id_to_label = dict(enumerate(labels["labels"].tolist(), 1))
        
        # convert dict to DF
        img_to_label = pd.DataFrame(image_id_to_label, index=[0]).T
        img_to_label["label"] = img_to_label[0]
        img_to_label["index"] = img_to_label.index
        del img_to_label[0]
        
        # splitting the data - using ramdom state for fixed split
        X_train, X_temp, y_train, y_temp = train_test_split(img_to_label[["index"]],img_to_label["label"],test_size=0.5,random_state=random_s)
        X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,test_size=0.5,random_state=random_s+1)

        # extract images file names & labels according to requested data set - train\validation\test
        image_ids, labels = (X_train, y_train) if self._split == 'train' else ((X_val, y_val) if self._split == 'val' else (X_test, y_test))
        image_ids["index"] = image_ids["index"].apply(lambda image_id: self._images_folder / f"image_{image_id:05d}.jpg")
        
        labels = labels - 1
        
        self._labels = labels.values
        self._image_files = image_ids['index'].values

    def __len__(self) -> int:
        return len(self._image_files)

    def __getitem__(self, idx) -> Tuple[Any, Any]:
        image_file, label = self._image_files[idx], self._labels[idx]
        image = PIL.Image.open(image_file).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.target_transform:
            label = self.target_transform(label)

        return image, label


    def extra_repr(self) -> str:
        return f"split={self._split}"

    def _check_integrity(self):
        if not (self._images_folder.exists() and self._images_folder.is_dir()):
            return False

        for id in ["label", "setid"]:
            filename, md5 = self._file_dict[id]
            if not check_integrity(str(self._base_folder / filename), md5):
                return False
        return True

    def download(self):
        if self._check_integrity():
            return
        download_and_extract_archive(
            f"{self._download_url_prefix}{self._file_dict['image'][0]}",
            str(self._base_folder),
            md5=self._file_dict["image"][1],
        )
        for id in ["label", "setid"]:
            filename, md5 = self._file_dict[id]
            download_url(self._download_url_prefix + filename, str(self._base_folder), md5=md5)


# preprocssing process

In [3]:
# train_data_mean = [0.4231, 0.3578, 0.2785]
# train_data_std = [0.3086, 0.2553, 0.2731]

train_transform = transforms.Compose([
    transforms.Resize((224,224)),
#     transforms.Resize((230,230)),
#     transforms.RandomRotation(30,),
#     transforms.RandomCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
#     transforms.Normalize(mean=train_data_mean, std=train_data_std)
])
    
valid_transform =  transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
#         transforms.Normalize(mean=train_data_mean, std=train_data_std)
])

# First random split

In [4]:
# Download training data from open datasets.
training_data = Flowers102(
    root="data",
    split="train",
    download=True,
    transform=train_transform
)

testing_data = Flowers102(
    root="data",
    split="test",
    download=True,
    transform=valid_transform
)

valid_data = Flowers102(
    root="data",
    split="val",
    download=True,
    transform=valid_transform
)


image_datasets = {'train':training_data, 'valid':valid_data, 'test':testing_data}
dataset_sizes = {i: len(image_datasets[i]) for i in ['train', 'valid','test']}
dataloaders = {i: DataLoader(image_datasets[i], batch_size=64, shuffle=True) for i in ['train', 'valid','test']}

In [5]:
# batch_size = 64

# # Create data loaders.
# train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)
# # test_dataloader = DataLoader(test_data, batch_size=batch_size)
# mean = torch.zeros(3)
# std = torch.zeros(3)
# print(len(training_data))
# for X, y in tqdm(train_dataloader):
# #     print(f"Shape of X [N, C, H, W]: {X.shape}")
# #     print(f"Shape of y: {y.shape} {y.dtype}")
    
# #     print((torch.mean(X, dim = [0,2,3])))
#     mean += torch.mean(X, dim = [0,2,3])
#     std += torch.std(X, dim = [0,2,3])
# #     print(a)
# #     print(a)
# #     break

# print(mean/64)
# print(std/64)
    
# dataloaders = {x: DataLoader(image_datasets[x], batch_size=64,
#                                              shuffle=True, num_workers=0) for x in ['train', 'valid','test']}

In [6]:
def train(model, criterion, optimizer, num_epochs, plot=True):
    epoch_train_loss = []
    epoch_train_acc = []
    epoch_valid_loss = []
    epoch_valid_acc = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
 
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
                append_to_loss = epoch_train_loss
                append_to_acc = epoch_train_acc
            else:
                model.eval()
                append_to_loss = epoch_valid_loss
                append_to_acc = epoch_valid_acc

            e_loss = 0
            e_correct = 0

            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                e_loss += loss.item() * inputs.size(0)
                e_correct += torch.sum(preds == labels.data)

            epoch_loss = e_loss / dataset_sizes[phase]
            epoch_acc = (e_correct.double() / dataset_sizes[phase]).item()

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            append_to_loss.append(epoch_loss)
            append_to_acc.append(epoch_acc)
            
    if plot:
        # Ploting graphs
        plt.plot(range(len(epoch_train_loss)), epoch_train_loss,
                 label='Training')
        plt.plot(range(len(epoch_valid_loss)), epoch_valid_loss,
                 label='Validation')
        plt.ylabel('Cross Entropy error')
        plt.xlabel('Epoch')
        plt.legend(loc='upper right')
        plt.show()

        plt.plot(range(len(epoch_train_acc)), epoch_train_acc,
                 label='Training')
        plt.plot(range(len(epoch_valid_acc)), epoch_valid_acc,
                 label='Validation')
        plt.ylabel('Accuracy')
        plt.xlabel('Epochs')
        plt.legend(loc='lower right')
        plt.show()
    
    return epoch_acc, model

In [7]:
def test(model):
    model.eval()
    res = 0
    for inputs, labels in tqdm(dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            res += torch.sum(preds.double() == labels.data)
            
    print(f"Test set accuracy: {res.item()}/{dataset_sizes['test']} -> {(res / dataset_sizes['test']).item()}")

## Hyper-parameter optimization on resnet model by simple grid search

In [ ]:
import itertools

lr = [0.05, 0.1, 0.15]
mom = [0.3, 0.9, 1]
ne = [3, 5, 10]
a = [lr,mom,ne]
best_acc = 0
best_hp = []
for (lr, momentum, e) in itertools.product(*a):
    model_resnet = models.resnet152(pretrained=True)
    resnet_fc_features = model_resnet.fc.in_features
    fc_resnet = nn.Sequential(
                nn.Linear(resnet_fc_features, 500),
                nn.ReLU(inplace=True),
                nn.Linear(500, 102)
            )
    
    model_resnet.fc = fc_resnet
    model_resnet = model_resnet.to(device)
    optimizer = optim.SGD(model_resnet.parameters(), lr=lr, momentum=momentum)
    criterion = nn.CrossEntropyLoss()
    acc, model_resnet = train(model_resnet, criterion, optimizer, num_epochs=e, plot=False)
    
    if acc > best_acc:
        best_acc = acc
        best_hp = [lr,momentum, e]

print(f'best_acc: {best_acc}, best_hp: {best_hp}')

# First model - resnet152

In [8]:
# replace last layer with new hidden layer & adjusted output layer

model_resnet = models.resnet152(pretrained=True)
resnet_fc_features = model_resnet.fc.in_features

fc_resnet = nn.Sequential(
            nn.Linear(resnet_fc_features, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_resnet.fc = fc_resnet
# model_resnet

## train and evaluate resnet model on first split

In [9]:
model_resnet = model_resnet.to(device)
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(model_resnet.parameters(), lr=0.1, momentum=0.3)
last_ac, model_resnet = train(model_resnet, criterion, optimizer, num_epochs=5)

test(model_resnet)

# Second model - vgg19_bn

In [10]:
# replace last layer with new hidden layer & adjusted output layer

model_vgg19 = models.vgg19_bn(pretrained=True)
vgg_clf_features = model_vgg19.classifier[0].in_features

clf_vgg = nn.Sequential(
            nn.Linear(vgg_clf_features, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_vgg19.classifier = clf_vgg
# model_vgg19

## train and evaluate vgg19 model on first split

In [11]:
model_vgg19 = model_vgg19.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_vgg19.parameters(), lr=0.1, momentum=0.3)
acc, model_vgg19 = train(model_vgg19, criterion, optimizer_ft, num_epochs=5)

test(model_vgg19)

# Second random split

In [19]:
# Download training data from open datasets.
training_data = Flowers102(
    root="data",
    split="train",
    download=True,
    transform=train_transform,
    random_s=456
)

testing_data = Flowers102(
    root="data",
    split="test",
    download=True,
    transform=valid_transform,
    random_s=456
)

valid_data = Flowers102(
    root="data",
    split="val",
    download=True,
    transform=valid_transform,
    random_s=456
)

image_datasets = {'train':training_data, 'valid':valid_data, 'test':testing_data}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid','test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=0)
               for x in ['train', 'valid','test']}

## restarting models

In [20]:
model_resnet = models.resnet152(pretrained=True)
resnet_fc_features = model_resnet.fc.in_features

fc_resnet = nn.Sequential(
            nn.Linear(resnet_fc_features, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_resnet.fc = fc_resnet
# model_resnet

model_vgg19 = models.vgg19_bn(pretrained=True)
vgg_clf_features = model_vgg19.classifier[0].in_features

clf_vgg = nn.Sequential(
            nn.Linear(vgg_clf_features, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_vgg19.classifier = clf_vgg
# model_vgg19

## train and evaluate resnet model on second split

In [21]:
model_resnet = model_resnet.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_resnet.parameters(), lr=0.1, momentum=0.3)
acc, model_resnet = train(model_resnet, criterion, optimizer_ft, num_epochs=5)

test(model_resnet)

## train and evaluate vgg19 model on second split

In [15]:
model_vgg19 = model_vgg19.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_vgg19.parameters(), lr=0.1, momentum=0.3)
acc, model_vgg19 = train(model_vgg19, criterion, optimizer_ft, num_epochs=5)

test(model_vgg19)

# trying bigger models

In [22]:
# Download training data from open datasets.
training_data = Flowers102(
    root="data",
    split="train",
    download=True,
    transform=train_transform,
)

testing_data = Flowers102(
    root="data",
    split="test",
    download=True,
    transform=valid_transform,
)

valid_data = Flowers102(
    root="data",
    split="val",
    download=True,
    transform=valid_transform,
)

image_datasets = {'train':training_data, 'valid':valid_data, 'test':testing_data}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid','test']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=64, shuffle=True, num_workers=0)
               for x in ['train', 'valid','test']}

# CNN with 2 hidden layers

In [23]:
model_resnet = models.resnet152(pretrained=True)
resnet_fc_features = model_resnet.fc.in_features

fc_resnet = nn.Sequential(
            nn.Linear(resnet_fc_features, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_resnet.fc = fc_resnet
# model_resnet

model_vgg19 = models.vgg19_bn(pretrained=True)
vgg_clf_features = model_vgg19.classifier[0].in_features

clf_vgg = nn.Sequential(
            nn.Linear(vgg_clf_features, 1000),
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Linear(500, 102)
        )

model_vgg19.classifier = clf_vgg
# model_vgg19

## train and evaluate resnet model

In [24]:
model_resnet = model_resnet.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_resnet.parameters(), lr=0.1, momentum=0.3)
acc, model_resnet = train(model_resnet, criterion, optimizer_ft, num_epochs=10)

test(model_resnet)

## train and evaluate vgg19 model

In [25]:
model_vgg19 = model_vgg19.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_vgg19.parameters(), lr=0.1, momentum=0.3)
acc, model_vgg19 = train(model_vgg19, criterion, optimizer_ft, num_epochs=10)

test(model_vgg19)